In [1]:
import pandas as pd
import nltk
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\veni\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
fullcorpus=pd.read_csv("SMSSpamCollection (1).tsv",sep="\t",header=None)
fullcorpus=fullcorpus.rename(columns={0:"title",1:"body"})
fullcorpus

,title,body
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...
5564,ham,Will ü b going to esplanade fr home?
5565,ham,"Pity, * was in mood for that. So...any other s..."
5566,ham,The guy did some bitching but I acted like i'd...


In [4]:
fullcorpus.shape

(5568, 2)

In [5]:
fullcorpus.title.value_counts()

ham     4822
spam     746
Name: title, dtype: int64

In [6]:
fullcorpus.isnull().sum()

title    0
body     0
dtype: int64

In [7]:
fullcorpus

,title,body
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...
5564,ham,Will ü b going to esplanade fr home?
5565,ham,"Pity, * was in mood for that. So...any other s..."
5566,ham,The guy did some bitching but I acted like i'd...


In [8]:
ps=nltk.PorterStemmer()
nltk.download('wordnet')
nltk.download('omw-1.4')
wnl=nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\veni\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\veni\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [9]:
def remove_punctuation(text):
    text="".join([char for char in text if char not in string.punctuation])
    return text

def tokenized_list(sentence):
    text_sentence=re.split('\W+',sentence)
    return text_sentence

def removing_stopwords(text):
    no_stopword = [char for char in text if char not in stopword]
    return no_stopword

def stemming_word(text):
    stemmed_word = [ps.stem(char) for char in text]
    return stemmed_word

def lemmatizing(word_list):
    lemmatized_word = [wnl.lemmatize(word) for word in word_list]
    return lemmatized_word

In [10]:
stopword=nltk.corpus.stopwords.words("english")
stopwords=stopword.copy()
fullcorpus["punc_removed_body"]=fullcorpus["body"].apply(lambda x: remove_punctuation(x))
fullcorpus["tokenized_body"]=fullcorpus["punc_removed_body"].apply(lambda x: tokenized_list(x.lower()))
fullcorpus["stopword_removed_list"]=fullcorpus["tokenized_body"].apply(lambda x:removing_stopwords(x))
fullcorpus["Stem_word"]=fullcorpus["stopword_removed_list"].apply(lambda x:stemming_word(x))
fullcorpus["lemmatized_words"]=fullcorpus["stopword_removed_list"].apply(lambda x:lemmatizing(x))

In [11]:
fullcorpus

,title,body,punc_removed_body,tokenized_body,stopword_removed_list,Stem_word,lemmatized_words
0,ham,I've been searching for the right words to tha...,Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[ive, searching, right, words, thank, breather...","[ive, search, right, word, thank, breather, pr...","[ive, searching, right, word, thank, breather,..."
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[free, entry, 2, wkly, comp, win, fa, cup, fin...","[free, entri, 2, wkli, comp, win, fa, cup, fin...","[free, entry, 2, wkly, comp, win, fa, cup, fin..."
2,ham,"Nah I don't think he goes to usf, he lives aro...",Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[nah, dont, think, goes, usf, lives, around, t...","[nah, dont, think, goe, usf, live, around, tho...","[nah, dont, think, go, usf, life, around, though]"
3,ham,Even my brother is not like to speak with me. ...,Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[even, brother, like, speak, treat, like, aids...","[even, brother, like, speak, treat, like, aid,...","[even, brother, like, speak, treat, like, aid,..."
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[date, sunday]","[date, sunday]","[date, sunday]"
...,...,...,...,...,...,...,...
5563,spam,This is the 2nd time we have tried 2 contact u...,This is the 2nd time we have tried 2 contact u...,"[this, is, the, 2nd, time, we, have, tried, 2,...","[2nd, time, tried, 2, contact, u, u, 750, poun...","[2nd, time, tri, 2, contact, u, u, 750, pound,...","[2nd, time, tried, 2, contact, u, u, 750, poun..."
5564,ham,Will ü b going to esplanade fr home?,Will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]","[ü, b, going, esplanade, fr, home]","[ü, b, go, esplanad, fr, home]","[ü, b, going, esplanade, fr, home]"
5565,ham,"Pity, * was in mood for that. So...any other s...",Pity was in mood for that Soany other suggest...,"[pity, was, in, mood, for, that, soany, other,...","[pity, mood, soany, suggestions]","[piti, mood, soani, suggest]","[pity, mood, soany, suggestion]"
5566,ham,The guy did some bitching but I acted like i'd...,The guy did some bitching but I acted like id ...,"[the, guy, did, some, bitching, but, i, acted,...","[guy, bitching, acted, like, id, interested, b...","[guy, bitch, act, like, id, interest, buy, som...","[guy, bitching, acted, like, id, interested, b..."


In [12]:
fullcorpus=fullcorpus.drop(["tokenized_body","punc_removed_body","stopword_removed_list","Stem_word","lemmatized_words"],axis=1)
data=fullcorpus.copy()

In [13]:
# Alternative method to clean dataset

In [14]:
def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    #text = [ps.stem(word) for word in tokens if word not in stopwords]
    text = [wnl.lemmatize(word) for word in tokens if word not in stopwords]   
    return text

In [15]:
# creating count vectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_counts = count_vect.fit_transform(data['body'])


In [16]:
dataset=pd.DataFrame(X_counts.toarray())
dataset

,0,1,2,3,4,5,6,7,8,9,...,8904,8905,8906,8907,8908,8909,8910,8911,8912,8913
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5563,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5564,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5565,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5566,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X = dataset.copy()
y = data.iloc[:,0].values
labelencoder_y=LabelEncoder()
y=labelencoder_y.fit_transform(y)


In [18]:
def model_buiding(x,y,model):
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 42)
    classifier = model
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    print(f'For the {classifier} model, the values are given below :')
    print(f'The confusion matrix of is:\n\n{cm}')
    accuracy=accuracy_score(y_test, y_pred)
    print(f'\nThe accuracy score of the is: {accuracy}\n')

In [19]:
model_buiding(X,y,GaussianNB())
model_buiding(X,y,LogisticRegression())

For the GaussianNB() model, the values are given below :
The confusion matrix of is:

[[853 112]
 [ 13 136]]

The accuracy score of the is: 0.8877917414721723

For the LogisticRegression() model, the values are given below :
The confusion matrix of is:

[[964   1]
 [ 23 126]]

The accuracy score of the is: 0.9784560143626571

